In [1]:
import pandas as pd
import numpy as np
import h5py
import math

from tokenizers import ByteLevelBPETokenizer

In [2]:
samples = pd.read_csv('Reviews.csv')

In [3]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(samples['Review'], vocab_size=16384, min_frequency=2)

In [4]:
samples['tokens'] = samples['Review'].apply(lambda x: tokenizer.encode(x).ids)
tokenized_length = samples['tokens'].apply(len).max()
samples['tokens'] = samples['tokens'].apply(lambda x: (([0]*tokenized_length)+x)[-tokenized_length:])
samples['tokens'] = samples['tokens'].apply(lambda x: np.array(x))

tokenized_length

3521

In [5]:
samples['reweighed'] = samples['tokens'].apply(lambda x: x*sum(x))
samples['scaled'] = samples['reweighed'].apply(lambda x: x/sum(x))

In [6]:
x = np.array(samples['scaled'].tolist()).astype(np.float32)
y = np.array(samples['Sentiment'].tolist()).astype(np.uint8)

In [7]:
file = h5py.File('xy.h5', 'w')

file.create_dataset('x', data=x)
file.create_dataset('y', data=y)

file.close()